# 1D NMR Processing and Display

a simplified environment for processing 1D Bruker NMR datasets with `SPIKE`

## Initialization

### Initialization of the environment

In [1]:
# following line used for debuging
from importlib import reload

In [2]:
# load all python and interactive tools
%matplotlib notebook
from spike.Interactive import INTER as I
from spike.File.BrukerNMR import Import_1D
from spike.Interactive.INTER import FileChooser, Phaser1D, Show


          SPIKE
    Version     : 0.99.3
    Date        : 11-03-2019
    Revision Id : 396
*** Importing  << zoom3D >> Failed ***
plugins loaded:
Bruker_NMR_FT,  Bucketing,  FTMS_calib,  Fitter,  Linear_prediction,  PALMA,  Peaks,  apmin,  bcorr,  bokeh_display,  fastclean,  gaussenh,  pg_sane,  rem_ridge,  sane,  sg,  test,  urQRd,  wavelet, 

spike.plugins.report() for a short description of each plugins
spike.plugins.report('module_name') for complete documentation on one plugin


### Choose the file

The `FileChooser()` tool creates a dialog box which allows to choose a file on your disk

- The optional `base` argument, starts the exploration on a given location

In [6]:
reload(I)

<module 'spike.Interactive.INTER' from '/Users/mad/spike/spike/Interactive/INTER.py'>

In [8]:
FC = FileChooser(base='/DATA/pharma', filetype='fid')

VBox(children=(HBox(children=(Label(value='Chosen dir:  /DATA/pharma'), Button(description='Up Directory', style=ButtonStyle()))), HBox(children=(Select(description='Choose Dir', layout=Layout(width='50%'), options=('/DATA/pharma/ARTEref_161123', '/DATA/pharma/LM1-199', '/DATA/pharma/LM1-200', '/DATA/pharma/LM1-200-restreint', '/DATA/pharma/SMARTE', '/DATA/pharma/SMARTE_donnees', '/DATA/pharma/spike', '/DATA/pharma/TP'), value='/DATA/pharma/ARTEref_161123'), Button(description='Set Directory', style=ButtonStyle()))), Select(description='Choose File', layout=Layout(width='50%'), options=(' ',), value=' '), Text(value='', description='File to load', layout=Layout(width='70%'))))

After the selection, the selected filename is found in `FC.file`

In [4]:
FC.file

''

### Import dataset

This is simply done with the `Import_1D()` tool, which returns a `SPIKE` object.

We store it into a variable, evaluating the variable show a summary of the dataset. 

In [5]:
d1 = Import_1D(FC.file)
d1

Exception:  : file not found

Many methods are attached to the `SPIKE` object, they can be either applied one by one, or piped by chaining them.

`display()` is used to display the dataset.

In the current set-up, the figure can be explored *(zoom, shift, resize, etc)* with the jupyter tools displayed  below the dataset.
The figure can also be saved as a `png` graphic file.
At anytime, the figure can be frozen by clicking on the blue button on the upper right corner.

In [ ]:
d1.set_unit('sec').display()

## Basic Processing

many processing methods are available, they can be either applied one by one, or piped by chaining them.

### Fourier Transform

In [ ]:
D1 = d1.copy() # copy the imported data-set to another object for processing
D1.apod_em(0.3).zf(2).ft_sim().bk_corr().apmin()  # chaining  apodisation - zerofill - FT - Bruker correction - autophase
D1.set_unit('ppm').display()  # chain  set to ppm unit - and display

## Advanced processing

### en cours

In [ ]:
PD1 = I.AvProc1D(filename=FC.file)

### rephasing
might be required - an interactive phaser is available

Use `scale` and `zoom` to tune the display; then use `P0, P1, pivot` to optimize the phase.

Once finished, click on `Apply correction`

In [ ]:
Phaser1D(PD1.data)

In [ ]:
Show(PD1.data)

## Save data-set
either as stand alone native SPIKE files, (there are other formats)

In [ ]:
PD1.data.save('example.gs1')

or as a `csv` text file, - in which case, it is probably better to remove the imaginary part, not useful there.

The file contains some basic informations in addition to the spectral data

In [ ]:
PD1.data.real().save_csv('example.csv')

In [ ]:
! head -15 example.csv